# Collect results
Here we perform MRI/US fusion using the trained VoxelMorph models and collect the results

In [2]:
from pathlib import Path

import numpy as np
import tensorflow as tf

from notebooks.utils import SmartDataGenerator, split_dataset, dice_coeff, prepare_model

Prepare dataset generators using a specific seed for repeatability

In [15]:
seed = 24052022
base_folder = Path(r"C:\Users\ML\Desktop\seba_preprocessed_fusion\numpy_160")
train_data, validation_data, test_data = split_dataset(base_folder, train_test_split=0.95, train_val_split=0.90, seed=seed)

size = (160, 160, 160)
train_generator = SmartDataGenerator(train_data, dim=size, batch_size=1, seed=seed)
validation_generator = SmartDataGenerator(validation_data, dim=size, batch_size=1, seed=seed)
test_generator = SmartDataGenerator(test_data, dim=size, batch_size=1, seed=seed)

Prepare storing function

In [27]:
from tqdm import tqdm
from skimage import measure

def create_and_store_results(generator, lambda_param, gamma_param):
    results = {
        "prostate_dice": [], "target_dice": [], "target_error": [],
        "prostate_dice_def": [], "target_dice_def": [], "target_error_def": [],
        "prostate_dice_pre": [], "target_dice_pre": [], "target_error_pre": []
    }
    for idx in tqdm(range(len(generator))):
        test_input, test_output, mr_targets, us_targets, _ = generator(idx)

        test_pred = model.predict(test_input)
        prostate_dice_val = dice_coeff(test_pred[2], test_output[2])
        results["prostate_dice"].append(prostate_dice_val)
        prostate_dice_def_val = dice_coeff(test_pred[2], test_input[2])
        results["prostate_dice_def"].append(prostate_dice_def_val)
        prostate_dice_def_val = dice_coeff(test_input[2], test_output[2])
        results["prostate_dice_pre"].append(prostate_dice_def_val)

        mr_targets = mr_targets[0]
        us_targets = us_targets[0]
        if mr_targets is None or us_targets is None:
            print("no matching targets here")
            continue

        for select_target in range(min(mr_targets.shape[-1], us_targets.shape[-1])):
            test_input_target = mr_targets[np.newaxis, ..., [select_target]]
            test_output_target = us_targets[np.newaxis, ..., [select_target]]
            test_pred_target = model.apply_transform(test_input[0], test_input[1], test_input_target)

            # in "error" we use /2 because images have an isotropic spacing of 0.5 mm, meaning that the distance is double its size

            # Dice between target in MR_def and US
            target_dice_val = dice_coeff(test_pred_target, test_output_target)
            target_error_val = np.linalg.norm((measure.centroid(test_pred_target) - measure.centroid(test_output_target)) / 2)
            results["target_dice"].append(target_dice_val)
            results["target_error"].append(target_error_val)

            # Dice between target in MR and MR_def
            target_dice_val = dice_coeff(test_pred_target, test_input_target)
            target_error_val = np.linalg.norm((measure.centroid(test_pred_target) - measure.centroid(test_input_target)) / 2)
            results["target_dice_def"].append(target_dice_val)
            results["target_error_def"].append(target_error_val)

            # Dice between target in MR and US
            target_dice_val = dice_coeff(test_input_target, test_output_target)
            target_error_val = np.linalg.norm((measure.centroid(test_input_target) - measure.centroid(test_output_target)) / 2)
            results["target_dice_pre"].append(target_dice_val)
            results["target_error_pre"].append(target_error_val)

    # convert everything to ndarray
    for k in results.keys():
        results[k] = np.array(results[k])

    # store results
    np.savez_compressed(f"../results/results_{lambda_param}_{gamma_param}", results)

### Run the evaluation

Run for a single model

In [ ]:
# instantiate the network and load the model
hyper = {
    'lambda_param': 5.0,
    'gamma_param': 3.0
}
model = prepare_model(inshape=size, lambda_param=hyper['lambda_param'], gamma_param=hyper['gamma_param'])

checkpoints_dir = Path(f"../models/test_{hyper['lambda_param']}_{hyper['gamma_param']}/checkpoints")
latest_checkpoint = tf.train.latest_checkpoint(checkpoints_dir)
model.load_weights(latest_checkpoint)

create_and_store_results(test_generator, hyper['lambda_param'], hyper['gamma_param'])

Run for all the models

In [ ]:
model = prepare_model(inshape=size, lambda_param=0, gamma_param=0)
for checkpoints_dir in Path(f"../models").iterdir():
    _, lambda_param, gamma_param = checkpoints_dir.name.split("_")
    latest_checkpoint = tf.train.latest_checkpoint(checkpoints_dir / "checkpoints")
    model.load_weights(latest_checkpoint)

    create_and_store_results(test_generator, lambda_param, gamma_param)